In [351]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [352]:
from lib.optimizers import *
from lib.functions import *
import numpy as np
import matplotlib.pyplot as plt

### Helper functions

In [353]:
_infty = 1e16
_zero = 1e-32

def remove_nans_infs_and_zeros(arr):
    arr = np.array(arr)
    arr[np.isnan(arr)] = _infty
    arr[np.isinf(arr)] = _infty
    arr[arr > _infty] = _infty
    arr[arr == 0] = _zero
    arr[arr < _zero] = _zero
    return arr

# A neural network

In [354]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [355]:
def F(x, p):
    assert x.shape == (2,)
    assert p.shape == (23,)
    
    W2, W3, W4, b2, b3, b4 = p[:4], p[4:10], p[10:16], p[16:18], p[18:21], p[21:23]
    W2 = W2.reshape((2, 2))
    W3 = W3.reshape((3, 2))
    W4 = W4.reshape((2, 3))
    
    assert W2.shape == (2, 2)
    assert W3.shape == (3, 2)
    assert W4.shape == (2, 3)
    assert b2.shape == (2,)
    assert b3.shape == (3,)
    assert b4.shape == (2,)
    
    a1 = x
    a2 = sigmoid(W2 @ a1 + b2)
    a3 = sigmoid(W3 @ a2 + b3)
    a4 = sigmoid(W4 @ a3 + b4)
    y = a4
    
    assert y.shape == (2,)
    
    return y

In [356]:
def f(p, X, Y):
    assert p.shape == (23,)
    
    res = np.array([np.linalg.norm(F(x, p) - y) for x, y in zip(X, Y)])
    return res

In [357]:
def grad_C(p, x, y):
    assert p.shape == (23,)
    assert x.shape == (2,)
    assert y.shape == (2,)
    
    W2, W3, W4, b2, b3, b4 = p[:4], p[4:10], p[10:16], p[16:18], p[18:21], p[21:23]
    W2 = W2.reshape((2, 2))
    W3 = W3.reshape((3, 2))
    W4 = W4.reshape((2, 3))
    
    assert W2.shape == (2, 2)
    assert W3.shape == (3, 2)
    assert W4.shape == (2, 3)
    assert b2.shape == (2,)
    assert b3.shape == (3,)
    assert b4.shape == (2,)
    
    a1 = x
    z2 = W2 @ a1 + b2
    a2 = sigmoid(z2)
    z3 = W3 @ a2 + b3
    a3 = sigmoid(z3)
    z4 = W4 @ a3 + b4
    a4 = sigmoid(z4)
    
    assert a4.shape == (2,)
    
    delta_4 = sigmoid(z4) * (1 - sigmoid(z4)) * (a4 - y)
    delta_3 = sigmoid(z3) * (1 - sigmoid(z3)) * (W4.T @ delta_4)
    delta_2 = sigmoid(z2) * (1 - sigmoid(z2)) * (W3.T @ delta_3)
    
    assert delta_4.shape == (2,)
    assert delta_3.shape == (3,)
    assert delta_2.shape == (2,)
    
    grad_W2 = delta_2.reshape((2, 1)) @ a1.reshape((1, 2))
    grad_W3 = delta_3.reshape((3, 1)) @ a2.reshape((1, 2))
    grad_W4 = delta_4.reshape((2, 1)) @ a3.reshape((1, 3))
    
    assert grad_W2.shape == (2, 2)
    assert grad_W3.shape == (3, 2)
    assert grad_W4.shape == (2, 3)
    
    grad_b2 = delta_2
    grad_b3 = delta_3
    grad_b4 = delta_4
    
    assert grad_b2.shape == (2,)
    assert grad_b3.shape == (3,)
    assert grad_b4.shape == (2,)
    
    grad = np.concatenate((grad_W2.flatten(), grad_W3.flatten(), grad_W4.flatten(), grad_b2, grad_b3, grad_b4))
    
    assert grad.shape == (23,)
    
    return grad

# Data

In [358]:
num_samples = 30
X = np.array([[x, x**2] for x in range(num_samples)])
Y = np.vstack((X[:, 0], X[:, 1])).T
X[:5, :], Y[:5, :]

(array([[ 0,  0],
        [ 1,  1],
        [ 2,  4],
        [ 3,  9],
        [ 4, 16]]),
 array([[ 0,  0],
        [ 1,  1],
        [ 2,  4],
        [ 3,  9],
        [ 4, 16]]))

In [359]:
f(np.random.randn(23), X, Y)

array([3.10020627e-01, 1.15715249e+00, 4.20760546e+00, 9.22806184e+00,
       1.62390497e+01, 2.52458023e+01, 3.62503554e+01, 4.92536285e+01,
       6.42560932e+01, 8.12580151e+01, 1.00259555e+02, 1.21260817e+02,
       1.44261870e+02, 1.69262761e+02, 1.96263525e+02, 2.25264188e+02,
       2.56264768e+02, 2.89265280e+02, 3.24265735e+02, 3.61266143e+02,
       4.00266510e+02, 4.41266841e+02, 4.84267143e+02, 5.29267418e+02,
       5.76267671e+02, 6.25267903e+02, 6.76268118e+02, 7.29268316e+02,
       7.84268501e+02, 8.41268673e+02])

In [360]:
def r(p):
    return f(p, X, Y)

def Dr(p):
    return np.vstack([grad_C(p, x, y) for x, y in zip(X, Y)])

N = X.shape[0]
M = 23
M, N

(23, 30)

In [361]:
R = Function(F=r, DF=Dr, M=23, N=N)

# Test different methods

In [362]:
np.random.seed(0)
p0 = 1 / np.sqrt(M) * np.random.randn(M)
max_iter = 1000
alpha_gn = 1e1
max_iter_gd = 10000
alpha_gd = 1e3
p0

array([ 0.36783034,  0.08343855,  0.20408098,  0.46725853,  0.38941276,
       -0.20377652,  0.19810713, -0.03156016, -0.02152262,  0.08561571,
        0.03003516,  0.30323699,  0.15868734,  0.02537099,  0.09255188,
        0.06957591,  0.31153702, -0.04277846,  0.06527913, -0.17809127,
       -0.53233518,  0.1362889 ,  0.18024741])

In [363]:
np.linalg.norm(R(p0))

2112.431985653707

## Stochastic gradient

In [364]:
p = p0

for epoch in range(max_iter_gd):
    i = np.random.randint(0, len(X))
    x, y = X[i,:], Y[i]
    p -= alpha_gd * grad_C(p, x, y)
    
err = np.linalg.norm(f(p, X, Y))
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


## Gradient descent

In [365]:
p, err = gradient_descent(R, p0, alpha_gd, max_iter_gd)
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


## Gauss-Newton

In [366]:
p, err = gauss_newton(R=R, p0=p0, max_iter=max_iter, alpha=alpha_gn, step_type="least_squares")
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


## Levenberg-Marquardt

#### 1. Constant $\lambda_k$ sequence

In [367]:
lambda_param = 1.
optimizer = LevenbergMarquardt(
    R=R, lambda_param_fun=LevenbergMarquardt.LambdaParamConstant(lambda0=lambda_param)
)

In [368]:
p, err = optimizer.optimize(p0, max_iter, step_type="least_squares")
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


In [369]:
p, err = optimizer.optimize(p0, max_iter, step_type="cgnr", step_max_iter=6, step_tol=1e-6)
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


In [370]:
p, err = optimizer.optimize(p0, max_iter, step_type="svd")
print(f"{p=}\n{err=}")

p=array([ 3.67830341e-01,  8.34385458e-02,  2.04080977e-01,  4.67258532e-01,
        8.50769018e+02,  8.50175829e+02,  3.41371968e+03,  3.41349001e+03,
        2.34160636e+03,  2.34171350e+03,  3.04818188e+03,  2.69460267e+03,
        2.08698922e+03,  7.29863970e+04,  6.45136567e+04,  4.99681146e+04,
        3.11537022e-01, -4.27784552e-02,  8.50444884e+02,  3.41334348e+03,
        2.34109555e+03,  5.42001919e+03,  1.29776388e+05])
err=2110.6003885150785


#### 2. Sequence $\lambda_k$ decrasing when error is decreasing, and increasing otherwise

In [371]:
lambda_param = 1.
lambda_change = 2.
optimizer = LevenbergMarquardt(
    R=R, lambda_param_fun=LevenbergMarquardt.LambdaParamDefaultModifier(lambda0=lambda_param, lambda_change=lambda_change)
)

In [372]:
p, err = optimizer.optimize(p0, max_iter, step_type="cholesky")
print(f"{p=}\n{err=}")

KeyboardInterrupt: 

In [ ]:
p, err = optimizer.optimize(p0, max_iter, step_type="cgnr", step_max_iter=6, step_tol=1e-6)
print(f"{p=}\n{err=}")

p=array([ 0.44364667, -0.34899506,  0.64851184,  1.52386946, -0.24480518,
       -4.18642183,  1.56485549, -2.97968555, -0.45877092,  8.77812663,
        0.20815047,  0.50293939, 13.11306863,  1.0453993 ,  2.47683831,
       50.98316778, -1.03334643,  0.31965958, -4.86292703, -5.16310465,
        9.70650825, 13.09335596, 53.59734248])
err=987.0992857863894


In [ ]:
p, err = optimizer.optimize(p0, max_iter, step_type="svd")
print(f"{p=}\n{err=}")

p=array([ 0.44364667, -0.34899506,  0.64851184,  1.52386946, -0.24480518,
       -4.18642183,  1.56485549, -2.97968555, -0.45877092,  8.77812663,
        0.20815047,  0.50293939, 13.11306863,  1.0453993 ,  2.47683831,
       50.98316778, -1.03334643,  0.31965958, -4.86292703, -5.16310465,
        9.70650825, 13.09335596, 53.59734248])
err=987.0992857863894
